In [ ]:
! pip uninstall tensorflow -y

In [ ]:
! pip install tensorflow-gpu==2.6

In [ ]:
import tensorflow as tf
tf.__version__

Importing all libreries we need.

In [ ]:

import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr
import seaborn as sns
import requests
!pip install openpyxl 


A little code that better managing the cach memory use by dictionary cell

In [ ]:
# Managinig the place dic in Cach memory 

cache = dict()
def get_article_from_server(url):
    print("Fetching article from server...")
    response = requests.get(url)
    return response.text
def get_article(url):
    print("Getting article...")
    if url not in cache:
        cache[url] = get_article_from_server(url)
    return cache[url]


Importing 3 tables 

In [ ]:
# Prepairing Datas @ Louding of Datas
# Prepairing Datas @ Louding of Datas
# Most of the work is to work on the Data.

x_features = pd.read_excel("../input/datas-features/Features data set.xlsx") # features 
y_sales = pd.read_excel('../input/datas-features/sales data-set.xlsx') # labels
z_stores = pd.read_csv("../input/datas-features/stores data-set.csv") # stores


In [ ]:
# Inquering all Datas

print(x_features.head())
print(y_sales.head())
print(z_stores.head())
print('-'*100)
print('\n')

"""
# After looking the Datas .
# I choose as best as I can the featurs that fit the economic aspects predict.
# Following this vision I don't take the data of the size stores - it doesn't fit the predict economic sales problem.
"""

In [ ]:
# The groupby drop all missing rows
# Merging all Datas
# Merging all Datas
# As an inner mean - marge just the common Datas.

y_sales['Date'] = pd.to_datetime(y_sales['Date'])
# optimize the sales via store in a unique date with all Deparments to be preper 
y_sales = y_sales.groupby(['Store', 'Date']).sum()
# we have to repeat its again to fit x_features y_sales 
x_features['Date'] = pd.to_datetime(x_features['Date'])

x_features = x_features.groupby(['Store', 'Date']).sum()


print(x_features.dtypes)
print('-'*100)
print('\n')
print(y_sales.dtypes)
print('-'*100)
print('\n')
print(z_stores.dtypes)
print('-'*100)
print('\n')

In [ ]:
Combined_table = pd.merge(x_features, y_sales['Weekly_Sales'], how='inner', right_index=True, left_index=True)
# Inqueries with code.
Combined_table.isna().sum()
Combined_table.info

Combined_table['Weekly_Sales'].describe()
# Very well... It seems that minimum price is larger than zero. Excellent!

In [ ]:
# Inquery with Plotting !
# ------------------------

#  --  First Plot as histogram plot.

In [ ]:
# 1) histogram plot
# let see how Weekly_Sales corellative to other features ?
# sns.distplot(Combined_table['Weekly_Sales'])
sns.displot(data = Combined_table, x='Weekly_Sales', kde=True)


In [ ]:
# 1) Deviate from the normal distribution.
# 2) Have appreciable positive skewness.
# 3) Have appreciable positive skewness.

print("Skewness: %f" % Combined_table['Weekly_Sales'].skew())
print("Kurtosis: %f" % Combined_table['Weekly_Sales'].kurt())


#  --  I try to inquery Datas by Skewness and Kurtosis 

#  --  Second plot as box plot

In [ ]:
# 2) box plot store / Weekly_Sales
# first convert the store from index into a column
# Here I Takea a look over each sotre outlayers via "Weekly_Sales'.
# It's other way to look the components behind the features.

Combined_table = Combined_table.reset_index(level=0)
var = 'Store'
data = pd.concat([Combined_table['Weekly_Sales'], Combined_table[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="Weekly_Sales", data=data)
fig.axis(ymin=180000, ymax=4000000)



#  -- Thierd plot - comparative features

In [ ]:
# 3) Plotting all the features via the label (prediction of sales)
Combind_graf = Combined_table.copy()
Combind_graf[['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Weekly_Sales']].plot(subplots=True, figsize=(20,15))
plt.show()

# After looking this plot there in no feature that could be pridictable for 'Weekly_Sales'.
# But at the end we will see that all features together could pridictable.





#  --  4th Confusionall Correlation metrix 

In [ ]:
# 4) Plotting Confusional Correlation matrix between numerical values
# In the following Matrix we will see how mach the features are confusing?.
# As we see the features are not corelative, except two. 
g = sns.heatmap(Combind_graf[['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Weekly_Sales']].corr(),annot=True, fmt = ".2f", cmap = "coolwarm")
plt.show()
#The Pearson accurecy is around 90%

#  --  5th Confusional Correlation matrix after reducing one of the colerative feature.

In [ ]:
# 5) Plotting Confusional Correlation matrix between numerical values with no features correlative
# Since I have two Prameters with correlasition, I drop 'MarkDown1', Why ?
# To prevent  Linkage featurs that are corelatived.
# Once I reduse feature the training forced being better.
g = sns.heatmap(Combind_graf[['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Weekly_Sales']].corr(),annot=True, fmt = ".2f", cmap = "coolwarm")
plt.show()

#Finally The linckage is no so big.




#  -- 6th Plotting all features 

In [ ]:
# 6) scatterplot
# let see what happend in with any feature 
sns.set()
cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Weekly_Sales']
sns_plot = sns.pairplot(Combind_graf[cols].sample(250), height = 2.5) 
plt.show()


print('Just to be on the safe side I take the correlation Pearson as a Metric')
"""
# =====================================================================================================
# Economic vision analization was taken  ----
# I analize only two table : features and sales arreies, to predict future weekly sales  growing.
# Union Datas here are with an economic vision and not blind merge Datas.
# We can later adding Stores and Some parmeters to mesure efficiency  per m*m

# The loss model is almost 10**11 (train and val was fitten) , mean that the model is good !!.
# No fitting and no under fitting.
# It is hardly train and practice small Datas. even that the prediction was done good. 
# In mean while when i Take Pearson correlation the correlation is around 88%-93%.
# prediction label is not 100%, mean no fiiting

# As a model of economic predicts, I can't do any augmantation with no meaning.
# just to "fix" Pearson correlation. 
# Even no augmantation Model Dence Deep Learning give us an excelent predicts and answers .


# ------------------------------------------------------------------------------------------------------
# =======================================================================================================
"""

In [ ]:

# Let see the first 10 variable

print("Dats shape = {}".format(Combined_table.shape))
print()
print("Lets see some feature:")
print(Combined_table[1:10])

Combined_table = Combined_table.drop(['MarkDown1'], axis=1)
 
# Definition of Y (label) and X (inputs)

# Define label for the new merge table: "combined_table"
# The 'Weekly_Sales' isIndexial so it dosn't take as a label
y = Combined_table['Weekly_Sales']
# Define features for the new merge table: "combined_table"
x = Combined_table.drop(['Weekly_Sales'], axis=1)



# Changing Farenhight to Celcius temperature.

Combined_table['Temperature'] = (Combined_table['Temperature']- 32) * 5./9.

print(Combined_table)
print('-'*100)
print('\n')

After that I filtering and Inquery we can go to preprocessing and Training.

Making NormalizationMaking Normalization

In [ ]:
# preprocessing normalization values between 0 and 1

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
y.shape
y = y.values.reshape(6435, 1)
y_scaled = scaler.fit_transform(y)

x.head()
x.tail()




Splitting the Datas in train, valid, and test.

In [ ]:
# splitting the Datas to train @ valid @ test 
# we need to take x_scaled after being notmalized
# In the first step we will split the data in training and remaining dataset
x_train, x_valid, y_train, y_valid = train_test_split(x_scaled, y, train_size=0.80, random_state=42)



# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
x_test, x_valid, y_test, y_valid = train_test_split(x_scaled,y, test_size=0.5)


# we need to add an additional dimantion to get numpy arrey to keras

x_train = x_train.reshape(x_train.shape[0],1,x_train.shape[1]) 
x_valid = x_valid.reshape(x_valid.shape[0],1,x_valid.shape[1])
x_test = x_test.reshape(x_test.shape[0],1,x_test.shape[1]) 


Test_Data = (x_test, y_test)

print('-'*100)
print('\n')


print(x_train.shape), print(y_train.shape)
print(x_valid.shape), print(y_valid.shape)
print(x_test.shape), print(y_test.shape)


print('-'*100)
print('\n')

Input prameters for LSTM Model

In [ ]:
# Training of the basic LSTM model
# Training of the basic LSTM model

# Initializing the Recurrent Neural Network AS LSTM
inputs = tf.random.normal([32, 50, 10])
model = Sequential()

#Adding the first LSTM layer with a sigmoid activation function and some Dropout regularization
#Units - dimensionality of the output space


model.add(LSTM(units = 32, return_sequences = False, input_shape =(1,x_train.shape[2])))
# Adding the output layer
model.add(Dense(units = 128))
model.add(Dense(units = 64))
model.add(Dense(units = 1, activation="relu", input_shape=(4,)))
model.summary()

Training of the basic LSTM model

Hyperparameters and then Model Dense (Deep Learning) 

In [ ]:
# ----- Hyperparameters
# learning rate need to be check !!
# Adam was config as Adaptive Learning Rate Methods
opt = tf.keras.optimizers.Adam(learning_rate = 0.1, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)
# model.compile(loss = 'mean_absolute_error', metrics=[soft_acc], optimizer=opt)
model.compile(loss = 'mse', optimizer=opt)

# A logger was created for logs the best whieghts of the training
my_callbacks = [tf.keras.callbacks.ModelCheckpoint(save_best_only=True, filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')]


# Create a TensorBoard logger need to becheck
logger = tf.keras.callbacks.TensorBoard(log_dir='logs', write_graph=True,
    histogram_freq=5)


history = model.fit(x_train,y_train,epochs = 1000, batch_size = 16, validation_data=(x_valid,y_valid), callbacks=my_callbacks)


p = history.history['loss']
print('-'*100)
print('\n')
# list all data in history
print(history.history.keys())


6th Plot of Training and Validation

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

plt.legend(['x_train', 'x_valid'], loc='upper left')
plt.legend(['y_train', 'y_valid'], loc='upper left')


#     PLOT Traing and Validation
# No Fitting, no under Fitting.

In [ ]:
# # ****   Prediction   *****
# # ****   Prediction   *****
# # ****   Prediction   *****



# verification of the prediction
# ===============================


# for Validation
preds_val = model.predict(x_valid)
preds_val = preds_val.squeeze()
result_val = y_valid - preds_val

# for Test
preds_test = model.predict(x_test)
preds_test = preds_test.squeeze()
result_test = y_test - preds_val


# Forecasting Accuracy = Pearson
# Spearman was checked for correlation, but Spearman is better for this problem.
# ------------------------------------------------------------------------------
# pearson correlation first way

def correlation_coefficient_var(y_valid, preds_coef_val):
    pearson_r_val = tfp.stats.correlation(preds_coef_val, y_valid)
    return(pearson_r_val)
    print(pearson_r_val)
    
    
def correlation_coefficient_test(y_test, preds_coef_test):
    pearson_r_test = tfp.stats.correlation(preds_coef_test, y_test)
    return(pearson_r_test)
    print(pearson_r_test)
    



print('-'*100)
print('\n')

preds_val = preds_val.reshape(-1, 1)
y_valid = y_valid.astype('float32')
print(correlation_coefficient_var(y_valid, preds_val))

preds_test = preds_test.reshape(-1, 1)
y_test = y_test.astype('float32')
print(correlation_coefficient_test(y_test, preds_test))

Prediction

In [ ]:
# 7) Plotting the label prediction of sales.
Combind_graf = Combined_table.copy()
Combind_graf[['Weekly_Sales']].plot(subplots=True, figsize=(20,15))
plt.title('model Weekly sales predict')
plt.ylabel('Income')
plt.xlabel('Weekly Time Sales')
plt.show()

7th plot of the 'weekly_Sales to comperative with the beginning plots

In [ ]:

(print(preds_val))
corr, p_val = pearsonr(y_valid.squeeze(), preds_val.squeeze())
print('Pearson corr validation')
print(corr)


(print(preds_test))
corr, p_test = pearsonr(y_test.squeeze(), preds_test.squeeze())
print('Pearson corr test')
print(corr)

#==============================================================================
# Via Pearson on Validation and Test verificatuin after  lots of epocs.
# WE could declair that the model prediction fits the Datas training 
# Aproximitly around 88% to 92% Pearson accurecy.
#==============================================================================
print('Via Pearson on Validation and Test verificatuin after lots of epocs.')
print('WE could declair')
    


"""
#                           Summary
# ==================================================================
# Finally we saw sequential Loss 'validation' via 'training' even:
# 1) All the faurues arn't correlative each on to others.                           
# 2) All features stores are with outlayers.                                    
# 3) Correlation graph are not a simetric Gaussian.                                 
# 4) Small Datas.                                                                    
# 5) Big numbers sales cause big differances complex calaulation.       
# Loss 'test' is sequential via 'validation'.
# so: 'Training'-->'Validation'-->'Test'---->sequentially. 
# ++++++++++++++++++++++++++  Conclussiotion  +++++++++++++++++++++++
#        Model: Dense Deep Learning with LSTM work very good  !!!!!!                 
# ===================================================================
# Prediction predict that the weekly sales will be the same'
# In the future as it were in the past in accurecy of 88%-93%.
# Base on this model !! looking via all features together.
# Else none of the features alone could be pridictable !!!.
# ===================================================================
"""